In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import matplotlib.pyplot as plt
import re
import urllib.request
import time
from transformer import create_padding_mask, PositionalEncoding, CustomMultiHeadAttention, CustomSchedule, accuracy, preprocess_sentence, evaluate, predict, transformer
import os


In [2]:
# urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [4]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

tokenizer.save_to_file('tokenizer')

In [5]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [6]:
MAX_LENGTH = 40

In [7]:
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in zip(inputs, outputs):
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    return tokenized_inputs, tokenized_outputs

In [8]:
questions, answers = tokenize_and_filter(questions, answers)


In [9]:

BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    },
))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [10]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [11]:
learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=[-1, tf.shape(y_pred)[1]])

    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])


In [12]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)


Epoch 1/50
185/185 [==============================] - 61s 299ms/step - loss: 1.4569 - accuracy: 0.0277
Epoch 2/50
185/185 [==============================] - 54s 289ms/step - loss: 1.1791 - accuracy: 0.0495
Epoch 3/50
185/185 [==============================] - 55s 295ms/step - loss: 1.0041 - accuracy: 0.0506
Epoch 4/50
185/185 [==============================] - 55s 294ms/step - loss: 0.9288 - accuracy: 0.0545
Epoch 5/50
185/185 [==============================] - 55s 295ms/step - loss: 0.8716 - accuracy: 0.0576
Epoch 6/50
185/185 [==============================] - 54s 293ms/step - loss: 0.8126 - accuracy: 0.0616
Epoch 7/50
185/185 [==============================] - 53s 288ms/step - loss: 0.7459 - accuracy: 0.0677
Epoch 8/50
185/185 [==============================] - 55s 295ms/step - loss: 0.6720 - accuracy: 0.0754
Epoch 9/50
185/185 [==============================] - 55s 296ms/step - loss: 0.5925 - accuracy: 0.0843
Epoch 10/50
185/185 [==============================] - 55s 295ms/step - l

In [18]:


# Define the path where you want to save the model
save_path = 'C:\\Users\\Home\\Desktop\\201835671'
save_file = os.path.join(save_path, 'transformer_chatbot_model.h5')

# Create the directory if it does not exist
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save the model
model.save(save_file)
print(f"Model saved to {save_file}")


Model saved to C:\Users\Home\Desktop\201835671\transformer_chatbot_model.h5


In [14]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence


In [15]:
def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)
def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence


In [17]:
output = predict("안녕?")


Input: 안녕?
Output: 안녕하세요 .
